In [1]:
# To make sure to be in the right Python virtual environment
import sys
print(sys.executable)

/home/kabbalow/datacraft/rust_for_data_science/.venv/bin/python


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import time

## Dataset definition and visualization

In [20]:
n_blobs = 5
X, y = make_blobs(n_samples=12345678, centers=n_blobs, cluster_std=0.7)#, random_state=0)

labels, counts = np.unique(y, return_counts=True)

print(X.shape)
print(X[: 10])
print(y[: 10])

# plt.scatter(X[:, 0], X[:, 1], c=y)
# plt.title("Blobs clusters")
# plt.show()

(12345678, 2)
[[-7.82328613  2.42113718]
 [-4.44008941 -2.98732603]
 [-7.74588054  2.4839822 ]
 [ 7.84130023 -2.07976411]
 [ 5.42499729 -0.74099223]
 [ 5.29137169 -1.60365924]
 [ 5.65112702 -0.29633151]
 [-8.24259754  2.96040107]
 [ 6.19963605 -1.50580933]
 [-4.57085324 -2.11836428]]
[4 2 4 1 1 1 1 4 1 2]


In [7]:
def stratif_train_test_split(X, y, test_size=0.2, train_size=None):

    # Parameters
    n_samples, _ = X.shape
    labels = np.unique(y)
    n_labels = len(labels)
    train_input = True
    if train_size == None:
        train_input = False

    X_train_set = []
    X_test_set = []
    y_train_set = []
    y_test_set = []

    # Constructs subsets
    sub_label_sets = dict(zip(labels, [[] for _ in range(n_labels)]))
    for i in range(n_samples):
        sub_label_sets[y[i]].append(X[i])

    # Constructs train and test sets
    sub_test_sets = dict(zip(labels, [[] for _ in range(len(labels))]))
    sub_train_sets = dict(zip(labels, [[] for _ in range(len(labels))]))

    for l in range(n_labels):
        sub_label_set = sub_label_sets[l]
        n_samples_sub_label_set = len(sub_label_set)
        test_prop = 0
        train_prop = 0
        for i in range(len(sub_label_set)):
            if test_prop <= test_size:
                sub_test_sets[l].append(sub_label_set[i])
                test_prop += 1 / n_samples_sub_label_set
                y_test_set.append(l)
            else:
                if train_input:
                    if train_prop > train_size:
                        break
                sub_train_sets[l].append(sub_label_set[i])
                train_prop += 1/ n_samples_sub_label_set
                y_train_set.append(l)

    for l in range(n_labels):
        X_train_set += sub_train_sets[l]
        X_test_set += sub_test_sets[l]

    #return np.array(train_set), np.array(test_set)
    return X_train_set, X_test_set, y_train_set, y_test_set

In [24]:
# Python with sklearn
start = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print("Python sklearn time :", time.time() - start)
print("Python sklearn result: train_set length is", len(X_train), ", test_set length is", len(X_test))

Python sklearn time : 4.711221933364868
Python sklearn result: train_set length is 8271604 , test_set length is 4074074


In [25]:
# Python with manuel implementation
start = time.time()
X_train, X_test, y_train, y_test = stratif_train_test_split(X, y, test_size=0.33)
print("Python sklearn time :", time.time() - start)
print("Python sklearn result: train_set length is", len(X_train), ", test_set length is", len(X_test))

Python sklearn time : 15.653326511383057
Python sklearn result: train_set length is 8271603 , test_set length is 4074075
